In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/thyroid-cancer-recurrence-prediction/Thyroid_Diff.csv")

In [ ]:
df.head()

In [ ]:
file_path = '/kaggle/input/thyroid-cancer-recurrence-prediction/Thyroid_Diff.csv'
data = pd.read_csv(file_path)

# Display the columns of the dataset to find the target column name
print(data.columns)



In [ ]:
# Step 1: Handle missing values
# Fill missing values with the median for numerical features and the most frequent value for categorical features
for column in data.columns:
    if pd.api.types.is_numeric_dtype(data[column]):
        data[column] = data[column].fillna(data[column].median())
    else:
        data[column] = data[column].fillna(data[column].mode()[0])


In [ ]:
# Step 2: Encode categorical variables
# Convert categorical variables to numeric using Label Encoding
from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in data.columns:
    if data[column].dtype == 'object':
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le

In [ ]:
# Step 3: Scale numerical features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_columns = data.select_dtypes(include=[np.number]).columns
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# Display the first few rows of the preprocessed dataset
print(data.head())

In [ ]:
# Step 4: Identify and set the target column
target_column = 'Recurred'  # Change this if your target column has a different name
if target_column not in data.columns:
    raise ValueError(f"The specified target column '{target_column}' is not found in the dataset. Please update the target column name.")

X = data.drop(target_column, axis=1)
y_continuous = data[target_column]


In [ ]:
# Step 5: Convert continuous target to categorical
bins = [-float('inf'), 0.5, 1.5, float('inf')]  # Define bins as per your data
labels = ['low', 'medium', 'high']  # Define labels for the bins
y_categorical = pd.cut(y_continuous, bins=bins, labels=labels)

In [ ]:
# Step 6: Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

In [ ]:
# Step 7: Train a classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Step 8: Make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
# Step 9: Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Save the preprocessed data and predictions to disk (optional)
X_train.to_csv('/kaggle/working/X_train.csv', index=False)
X_test.to_csv('/kaggle/working/X_test.csv', index=False)
y_train.to_csv('/kaggle/working/y_train.csv', index=False)
y_test.to_csv('/kaggle/working/y_test.csv', index=False)
pd.DataFrame(y_pred, columns=['predictions']).to_csv('/kaggle/working/y_pred.csv', index=False)

print("Preprocessing, training, and prediction complete. Data and predictions have been saved.")
